# Extract all fixations (and normalize them)

In [275]:
%matplotlib inline
from matplotlib import pyplot as plt
import os
import numpy as np
import scipy
import glob
import sys
from scipy import io
import math
from skimage import transform
import random
import keras
from sklearn import preprocessing
import tqdm

# GLOBALS

In [276]:
POET_DIR = "D:/data/POETdataset/"
pascal_images = os.path.join(POET_DIR, 'PascalImages')
CLASS2IDX_FILE = os.path.join(POET_DIR, 'class2idx.npy')
IDX2CLASS_FILE = os.path.join(POET_DIR, 'idx2class.npy')
PATCH_SIZE = 78
HALF_PATCH = PATCH_SIZE//2
DATASET_SIZE = 6270
VALIDATION_PERC = 0.2
# MAX_GAZE_POINTS = 3 # TEMPORARY
MIN_GAZE_POINTS = 2
BATCH_SIZE = 50
os.chdir(POET_DIR)

In [277]:
os.chdir(POET_DIR)

In [278]:
classes = [a.split("_")[1].split(".mat")[0] for a in glob.glob("etData/*")]
idx2class = {i:c for i, c in enumerate(classes)}
class2idx = {c:i for i, c in enumerate(classes)}

In [279]:
def fixation_within_image(fx, fy, dims):
#   print(dims)
  if fx < 0:
    return False
  if fx > dims[1]:
    return False
  if fy < 0:
    return False
  if fy > dims[0]:
    return False

  return True

In [280]:
def get_avg_fixations(fixR, fixL, dims):
    # ger average of fixations between right and left eye
    # manage nans
    fix = []
    for i in range(len(fixR)):
        fR = fixR[i]
        fL = fixL[i]
        fx = None
        fy = None
        # no fixations outside
        if not np.isnan(fR[0]) and not np.isnan(fL[0]):
            fx = np.mean([fR[0],fL[0]])
        else:
            if np.isnan(fR[0]):
                fx = fL[0]
            elif np.isnan(fL[0]):
                fx = fR[0]
                
        if not np.isnan(fR[1]) and not np.isnan(fL[1]):
            fy = np.mean([fR[1],fL[1]])
        else:
            if np.isnan(fR[1]):
                fy = fL[1]
            elif np.isnan(fL[1]):
                fy = fR[1]
        if fixation_within_image(fx, fy, dims):
            fix.append([fx,fy])

    fix = np.array(fix)
    return fix

In [281]:
class_matrices = {}

In [294]:
def get_fixations(filename, classname, dims):
    filename = filename.split("%s_" %classname)[1]
    filename = filename.split('.')[0]
    if classname not in class_matrices.keys():
        c_instances = io.loadmat(os.path.join(POET_DIR,'etData','etData_%s.mat' %classname), squeeze_me=True)['etData']
        class_matrices[classname] = c_instances
    else:
        c_instances = class_matrices[classname]

    for i in c_instances:
        if filename == i['filename']:
            # get fastest user with at least X fixations
            sorted_indexes = np.argsort(i['rt'])
            for u_i in sorted_indexes:
                fixR = i['fixations'][u_i]['imgCoord']['fixR'].tolist()['pos'].tolist().tolist()
                fixL = i['fixations'][u_i]['imgCoord']['fixL'].tolist()['pos'].tolist().tolist()
                if len(fixR) > 0 and len(fixL) > 0:
                    if type(fixR[0]) is list and type(fixL[0]) is list:
                        fix = get_avg_fixations(fixR, fixL, dims)
                        if len(fix) >= MIN_GAZE_POINTS:
                            for f_point in fix:
                                f_point[0] = f_point[0]/dims[1]
                                f_point[1] = f_point[1]/dims[0]
                            return fix

In [295]:
import pickle

In [326]:
train_ids = pickle.load(open("train_ids.pkl",'rb'))

In [334]:
test_ids = pickle.load(open('test_ids.pkl', 'rb'))

In [335]:
len(test_ids)

614

In [332]:
train_ids[:10], test_ids[:10]

(['horse_2010_004477.jpg',
  'cat_2010_001351.jpg',
  'horse_2008_006434.jpg',
  'dog_2010_002586.jpg',
  'cat_2008_007239.jpg',
  'cat_2010_002305.jpg',
  'cat_2009_003419.jpg',
  'sofa_2008_004670.jpg',
  'cow_2009_003189.jpg',
  'dog_2010_002808.jpg'],
 ['sofa_2009_001376.jpg',
  'diningtable_2010_002880.jpg',
  'motorbike_2008_004554.jpg',
  'cow_2010_003345.jpg',
  'cat_2008_002668.jpg',
  'dog_2008_003665.jpg',
  'dog_2011_001543.jpg',
  'cat_2010_001457.jpg',
  'dog_2010_000395.jpg',
  'cat_2010_004816.jpg'])

In [299]:
filenames[:10]

['D:/data/POETdataset/PascalImages\\aeroplane_2008_000021.jpg',
 'D:/data/POETdataset/PascalImages\\aeroplane_2008_000033.jpg',
 'D:/data/POETdataset/PascalImages\\aeroplane_2008_000037.jpg',
 'D:/data/POETdataset/PascalImages\\aeroplane_2008_000064.jpg',
 'D:/data/POETdataset/PascalImages\\aeroplane_2008_000151.jpg',
 'D:/data/POETdataset/PascalImages\\aeroplane_2008_000197.jpg',
 'D:/data/POETdataset/PascalImages\\aeroplane_2008_000251.jpg',
 'D:/data/POETdataset/PascalImages\\aeroplane_2008_000291.jpg',
 'D:/data/POETdataset/PascalImages\\aeroplane_2008_000367.jpg',
 'D:/data/POETdataset/PascalImages\\aeroplane_2008_000585.jpg']

## Training data

In [300]:
classes

['aeroplane',
 'bicycle',
 'boat',
 'cat',
 'cow',
 'diningtable',
 'dog',
 'horse',
 'motorbike',
 'sofa']

In [301]:
full_fn

'D:\\data\\POETdataset\\PascalImages\\cat_2008_005460.jpg'

In [303]:
train_fix = []

In [305]:
for fn in tqdm.tqdm(train_ids):
    full_fn = os.path.abspath(os.path.join(POET_DIR, "PascalImages", fn))
    for cname in classes:
        if cname in full_fn:
            img_src = keras.preprocessing.image.img_to_array(
                keras.preprocessing.image.load_img(full_fn)
            )
            fix = get_fixations(full_fn, cname, (img.shape[0],img.shape[1]))
            train_fix.append(fix)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5517/5517 [00:32<00:00, 168.74it/s]


In [310]:
pickle.dump(train_fix, open("train_fix.pkl", 'wb'))

In [336]:
test_fix = []

In [337]:
for fn in tqdm.tqdm(test_ids):
    full_fn = os.path.abspath(os.path.join(POET_DIR, "PascalImages", fn))
    for cname in classes:
        if cname in full_fn:
            img_src = keras.preprocessing.image.img_to_array(
                keras.preprocessing.image.load_img(full_fn)
            )
            fix = get_fixations(full_fn, cname, (img.shape[0],img.shape[1]))
            test_fix.append(fix)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 614/614 [00:03<00:00, 156.84it/s]


In [338]:
pickle.dump(test_ids, open("test_fix.pkl", 'wb'))

In [1]:
for f in test_fix:
    if len(f) == 0:
        print("go fuck yourself")
    

NameError: name 'test_fix' is not defined

In [341]:
flat_list = [item for sublist in test_fix for item in sublist]


TypeError: 'NoneType' object is not iterable

In [340]:
np.max(test_fix)

AttributeError: 'list' object has no attribute 'tolist'